In [1]:
import ibis
from pyspark.sql import SparkSession

In [2]:

session = SparkSession.builder.getOrCreate()
ipsc = ibis.pyspark.connect(session)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/24 21:01:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/24 21:01:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# df_rdd = session.read.csv("../data/sample_big_mortgage_file.csv.gz", header=True, sep="|")

df_rdd = session.read.csv("../data/2020_lar.txt.gz", header=True, sep="|")

In [4]:
# df_rdd.count()

In [5]:
df_rdd.createTempView("tmp_df_table")

22/09/24 21:01:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
ipsc

In [7]:
ipsc.list_databases()

['default']

In [8]:
ipsc.list_tables()


['tmp_df_table']

In [9]:
my_table = ipsc.table("tmp_df_table")

In [10]:
for col in my_table.columns:
    if col[0] == "d":
        print(col)

derived_msa_md
derived_loan_product_type
derived_dwelling_category
derived_ethnicity
derived_race
derived_sex
discount_points
debt_to_income_ratio
denial_reason_1
denial_reason_2
denial_reason_3
denial_reason_4


In [13]:
query = "SELECT * FROM tmp_df_table LIMIT 5"

In [16]:
query = "SELECT DISTINCT derived_sex FROM tmp_df_table"

In [17]:
df1 = session.sql(query)

In [18]:
df1.show()

+-----------------+
|      derived_sex|
+-----------------+
|Sex Not Available|
|           Female|
|            Joint|
|             Male|
+-----------------+



In [ ]:
df_rdd.count()

In [19]:
### Now the Ibis approach

ibis.options.interactive = True

In [21]:
ipsc.current_database

'default'

In [22]:
ipsc.exists_table("tmp_df_table")

/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_30663/3364586496.py:1: FutureWarning: `exists_table` is deprecated as of v2.0; use `name in client.list_tables()`
  ipsc.exists_table("tmp_df_table")


True

In [23]:
ipsc.table("tmp_df_table").columns

['activity_year',
 'lei',
 'derived_msa_md',
 'state_code',
 'county_code',
 'census_tract',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_ethnicity',
 'derived_race',
 'derived_sex',
 'action_taken',
 'purchaser_type',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'reverse_mortgage',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'interest_rate',
 'rate_spread',
 'hoepa_status',
 'total_loan_costs',
 'total_points_and_fees',
 'origination_charges',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'multifamily_affordable_units',
 'in

In [30]:
t = ipsc.table("tmp_df_table")

In [31]:
expr = t.count()

In [ ]:
ibis.show_sql(expr)